In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import torch
import random
device = 'cuda' if torch.cuda.is_available() else 'cpu'
import os,sys
opj = os.path.join
from tqdm import tqdm
import acd
from copy import deepcopy
from model_mnist import LeNet5
from visualize import *
import dset_mnist as dset
import foolbox
sys.path.append('../trim')
from transforms_torch import transform_bandpass, tensor_t_augment, batch_fftshift2d, batch_ifftshift2d
from trim import *
from util import *
from attributions import *
from captum.attr import *
import warnings
warnings.filterwarnings("ignore")
# disentangled vae
sys.path.append('../disentangling-vae')
from collections import defaultdict
import vae_trim, vae_trim_viz
from disvae.utils.modelIO import save_model, load_model, load_metadata
from disvae.models.losses import get_loss_f

### Train model

In [2]:
args = vae_trim.parse_arguments()
args.btcvae_A = 0
args.btcvae_B = 9
args.reg_anneal = 0
args.trim_lamb = 50
args.epochs = 60
name = "disvae_btcvae_A_" + str(args.btcvae_A) + "_btcvae_B_" + str(args.btcvae_B) + "_lamb_" + str(args.trim_lamb) 
args.name = name

# load classifier
m = LeNet5().eval()
m.load_state_dict(torch.load('weights/lenet_epoch=12_test_acc=0.991.pth'))
m = m.to(device)

In [3]:
# train and evaluate model
vae_trim.main(args, classifier=m)

14:20:27 INFO - main: Root directory for saving and loading experiments: results/disvae_btcvae_A_0_btcvae_B_9_lamb_50
14:20:27 INFO - main: Train mnist with 60000 samples
14:20:27 INFO - main: Num parameters in model: 469173
14:20:27 INFO - __init__: Training Device: cuda
14:20:59 INFO - __call__: Epoch: 1 Average loss per image: -68.07
14:21:29 INFO - __call__: Epoch: 2 Average loss per image: -100.94
14:22:00 INFO - __call__: Epoch: 3 Average loss per image: -105.20
14:22:31 INFO - __call__: Epoch: 4 Average loss per image: -107.54
14:23:02 INFO - __call__: Epoch: 5 Average loss per image: -109.03
14:23:32 INFO - __call__: Epoch: 6 Average loss per image: -110.11
14:24:03 INFO - __call__: Epoch: 7 Average loss per image: -111.02
14:24:35 INFO - __call__: Epoch: 8 Average loss per image: -111.86
14:25:07 INFO - __call__: Epoch: 9 Average loss per image: -112.58
14:25:38 INFO - __call__: Epoch: 10 Average loss per image: -112.96
14:26:07 INFO - __call__: Epoch: 11 Average loss per imag

In [4]:
# generate visualization
args = vae_trim_viz.parse_arguments()
args.name = name
args.plots = "all"
args.n_rows = 10
args.n_cols = 10
vae_trim_viz.main(args)

Selected idcs: [50997, 28883, 7657, 490, 5940, 59701, 52887, 38156, 2288, 44011, 45422, 5500, 6450, 50232, 23241, 15519, 1142, 2019, 51693, 1038, 22681, 42488, 40847, 31735, 40358, 30379, 9735, 5982, 11999, 46754, 7498, 55400, 958, 32970, 31899, 57702, 16372, 4231, 43729, 35460, 59789, 30533, 4493, 39417, 44245, 5828, 33753, 37945, 3012, 17667, 54026, 36466, 4133, 42246, 19819, 31525, 55717, 23280, 17462, 16328, 42957, 13109, 29713, 57445, 34744, 43608, 1264, 33298, 4626, 378, 21856, 9422, 46531, 30987, 43080, 24729, 10951, 3550, 4996, 38504, 32543, 10748, 4936, 36525, 14096, 9453, 1777, 22690, 50526, 7267, 34713, 9255, 43942, 20014, 29762, 2594, 27279, 18139, 43410, 52855]


### Test model

In [5]:
args = vae_trim.parse_arguments()
args.name = name
# results dir
exp_dir = os.path.join(vae_trim.RES_DIR, args.name)

# load dataloaders
train_loader, test_loader = dset.load_data(args.batch_size, args.eval_batchsize, device)
metadata = load_metadata(exp_dir)

# load model
model = load_model(exp_dir, is_gpu=not args.no_cuda)

In [6]:
# loss
loss_f = get_loss_f(args.loss,
                    n_data=len(test_loader.dataset),
                    device=device,
                    **vars(args))

# evaluate on testset
storer = defaultdict(list)
for data, _ in tqdm(test_loader, leave=False, disable=args.no_progress_bar):
    data = data.to(device)
    recon_batch, latent_dist, latent_sample = model(data)
    _ = loss_f(data, recon_batch, latent_dist, model.training,
                    storer, latent_sample=latent_sample)    
    losses = {k: sum(v) / len(test_loader) for k, v in storer.items()}
    break
print(losses)

{'recon_loss': 15.124087524414062, 'loss': -14.043325805664063, 'mi_loss': 6.861013031005859, 'tc_loss': -6.017551422119141, 'dw_kl_loss': 0.07688323259353638, 'kl_loss': 0.6955610275268554, 'kl_loss_0': 0.18338215351104736, 'kl_loss_1': 0.13812520503997802, 'kl_loss_2': 0.06167691946029663, 'kl_loss_3': 0.00018223549704998732, 'kl_loss_4': 9.279078803956509e-05, 'kl_loss_5': 0.00010799762094393373, 'kl_loss_6': 0.14513283967971802, 'kl_loss_7': 7.482132059521973e-05, 'kl_loss_8': 0.16666001081466675, 'kl_loss_9': 0.00012611874844878913}
